In [1]:
import math
import time
import pyvisa as visa

from dp832.DP832 import DP832
from dmm6500.DMM6500 import DMM6500
from dmm6500.Sense import Function
from dmm6500.Sense import Range
from dmm6500.Trace import Element
from utils.Units import CurrentUnit

import pandas as pd
import plotly.express as px


In [2]:
rm = visa.ResourceManager()
failed = False
try:
    dp832 = DP832(rm)
    dp832.open('TCPIP::192.168.252.18::INSTR')
    dp832.clear()
    print(dp832.query_id())

    dmm = DMM6500(rm)
    dmm.open('TCPIP::192.168.252.20::INSTR')
    dmm.clear()
    dmm.reset()
    print(dmm.query_id())


    # Power Supply - Turn off timer so we can set configuration
    dp832.timer.turn_on(False)

    # Power Supply - Set OCP and OVP for channel 1
    dp832.output.overvoltage_protection_value(1, 4.9)
    dp832.output.overvoltage_protection(1, True)
    dp832.output.overcurrent_protection_value(1, 2.5)
    dp832.output.overcurrent_protection(1, True)

    # Power Supply - Select channel 1, set Timer configuration
    seconds_timer = 45
    dp832.instrument.nselect(1)
    dp832.timer.cycles(1)
    dp832.timer.groups(4)
    dp832.timer.parameter(0, 4.2, 2, seconds_timer)
    dp832.timer.parameter(1, 3.8, 2, seconds_timer)
    dp832.timer.parameter(2, 3.2, 2, seconds_timer)
    dp832.timer.parameter(3, 3.1, 2, seconds_timer)
    dp832.timer.turn_off_when_done(True)

    # DMM - Configure to measure Current DC
    read_seconds = math.ceil(seconds_timer * 4 * 1.05) # Add 5% to read time since we start reading before turning on power supply
    dmm.sense.function(Function.CURRENT_DC)
    dmm.sense.range(Function.CURRENT_DC, Range.CURRENT_DC_1A)
    dmm.trigger.duration_loop(read_seconds, 0, "defbuffer1") # Measure for requested seconds

    # DMM - Start measuring
    dmm.init() # Start the trigger model
    dmm.wait() # Wait for the trigger model to complete

    # Power Supply - Turn on channel 1 and enable timer
    dp832.output.turn_on(1, True)
    dp832.timer.turn_on(True)

    # Wait for Power Supply and DMM to be done running their scripts
    time.sleep(read_seconds)

    # Power Supply - Reset setting, OCP and OVP to a value I normally
    dp832.output.overvoltage_protection_value(1, 5.2)
    dp832.output.overcurrent_protection_value(1, 1.2)
    dp832.source.voltage(1, 4.2)
    dp832.source.current(1, 1.0)

    # DMM - Read stats
    max_read_data = dmm.trace.actual("defbuffer1")
    dmm_values = dmm.trace.data(max_read_data, "defbuffer1", Element.READING).split(',')
    dmm_timestamps = dmm.trace.data(max_read_data, "defbuffer1", Element.RELATIVE).split(',')
    for i in range(0, len(dmm_timestamps)):
        dmm_timestamps[i] = float(dmm_timestamps[i])
        dmm_values[i] = CurrentUnit.MILLI.convert(float(dmm_values[i]))

    dmm_stats_avg = dmm.trace.stats_average("defbuffer1")
    dmm_stats_min = dmm.trace.stats_min("defbuffer1")
    dmm_stats_max = dmm.trace.stats_max("defbuffer1")

    # Close sessions/connections
    dp832.close()
    dmm.close()

except visa.VisaIOError as err:
    print("Failed to open resource. Error: ", err)
    failed = True

Failed to open resource. Error:  VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.


In [3]:
d = {"Time": dmm_timestamps, "Reading": dmm_values}
df = pd.DataFrame(d)

NameError: name 'dmm_timestamps' is not defined

In [ ]:
fig = px.line(df, x="Time", y="Reading", title="My Readings")
fig.update_traces(mode="markers+lines")
fig.update_yaxes(type='linear')
fig.update_layout(
    yaxis = dict(
        ticksuffix = ' ' + CurrentUnit.MILLI.unit()
    ))
fig.show()

print(f"Max {CurrentUnit.MILLI.convert(dmm_stats_max)}mA")
print(f"Min {CurrentUnit.MICRO.convert(dmm_stats_min)}uA")
print(f"Avg {CurrentUnit.MILLI.convert(dmm_stats_avg)}mA")
